In [11]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from dateutil.relativedelta import relativedelta
from helper import *

# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

import pickle
from dateutil.relativedelta import relativedelta
import gc
from re import finditer

## Find DD DU
from helper import preprocess_data
from patterns import find_drawdowns, find_drawups

## MATCHING
import name_matching
from name_matching import name_match
import transaction_matching
from transaction_matching import transaction_match

## CONSOLIDATION
from consolidation import combine_matches, consolidate_matches, find_attritions, find_new_accounts, get_attrition_status, get_new_account_status

In [2]:
start_date = dataiku.get_custom_variables()['start_date']
end_date = dataiku.get_custom_variables()['end_date']

consistency = int(dataiku.get_custom_variables()['consistency'])
drawdown_period_average = int(dataiku.get_custom_variables()['drawdown_period_average'])
drawdown = int(dataiku.get_custom_variables()['drawdown'])
drawdown_fwd_check = int(dataiku.get_custom_variables()['drawdown_fwd_check'])
drawdown_lookback_period = int(dataiku.get_custom_variables()['drawdown_lookback_period'])
drawup_lookfwd_period = int(dataiku.get_custom_variables()['drawup_lookfwd_period'])
statistics_period = int(dataiku.get_custom_variables()['statistics_period'])
inactive_period = int(dataiku.get_custom_variables()['inactive_period'])

## MATCHING VARIABLES
month_diff_h = int(dataiku.get_custom_variables()['month_diff_h'])
month_diff_l = int(dataiku.get_custom_variables()['month_diff_l'])
sd_mul = int(dataiku.get_custom_variables()['sd_mul'])
max_city_distance = int(dataiku.get_custom_variables()['max_city_distance'])
threshold_score_step1 = int(dataiku.get_custom_variables()['threshold_score_step1'])
threshold_score_step2 = int(dataiku.get_custom_variables()['threshold_score_step2'])

## RUN TYPE
run = dataiku.get_custom_variables()['run_type']

print("start_date", start_date)
print("end_date", end_date)

start_date 2019-01-01
end_date 2022-10-01


In [3]:
# Read recipe inputs
NAFCUSTOMER_ACTIVE_CARDS_FULL = dataiku.Dataset("NAFCUSTOMER_ACTIVE_CARDS_FULL")
NAFCUSTOMER_ACTIVE_CARDS_FULL_df = NAFCUSTOMER_ACTIVE_CARDS_FULL.get_dataframe()
print(len(NAFCUSTOMER_ACTIVE_CARDS_FULL_df))
NAFCUSTOMER_ACTIVE_CARDS_FULL_df.head()

16505529


,CUSTOMER,REVENUE_YEAR,REVENUE_MONTH,ACTIVE_CARD_COUNT
0,ENCORE DERMATOLOGY INC,2019,1,32.0
1,BRADANICK CONSTRUCTION SERVICES INC,2019,1,3.0
2,PEREZ INDUSTRIES INC,2019,1,2.0
3,A S CAR CLEAN LLC,2019,1,0.0
4,M SULLIVAN AND SON LIMITED,2019,1,20.0


In [4]:
def date_tz_naive(pd_s):
    return pd.to_datetime(pd_s).apply(lambda x:x.tz_localize(None))

# Read recipe inputs
NAFCUSTOMER_ACTIVE_CARDS_FULL = dataiku.Dataset("NAFCUSTOMER_ACTIVE_CARDS_FULL")
NAFCUSTOMER_ACTIVE_CARDS_FULL_df = NAFCUSTOMER_ACTIVE_CARDS_FULL.get_dataframe()

print(len(NAFCUSTOMER_ACTIVE_CARDS_FULL_df))
NAFCUSTOMER_ACTIVE_CARDS_FULL_df.head()

16505529


,CUSTOMER,REVENUE_YEAR,REVENUE_MONTH,ACTIVE_CARD_COUNT
0,ENCORE DERMATOLOGY INC,2019,1,32.0
1,BRADANICK CONSTRUCTION SERVICES INC,2019,1,3.0
2,PEREZ INDUSTRIES INC,2019,1,2.0
3,A S CAR CLEAN LLC,2019,1,0.0
4,M SULLIVAN AND SON LIMITED,2019,1,20.0


In [8]:
customer_list = ['NORTHSHORE INC','JAYHAWK MILLWRIGHT','PAXTON ASSOCIATES INC','STATE OF NEW YORK',
                'PAXTON ASSOCIATES INC', 'YNGRID COSMETICZ LLC', 'CYRGUS COMPANY INC.', 'THE HEALTHY STOP INC',
                 'T F WALZ INC', 'JAYHAWK MILLWRIGHT', 'CREDIT SLAYERS LLC', 'A ABLE MOVING CO', 'STUDIO IMPACT INC']

df_v = NAFCUSTOMER_ACTIVE_CARDS_FULL_df[NAFCUSTOMER_ACTIVE_CARDS_FULL_df.CUSTOMER.isin(customer_list)]
print(len(df_v.CUSTOMER.unique()), "unique customers")
print(len(df_v), "records")

7 unique customers
165 records


In [13]:
df_v.head()

,CUSTOMER,REVENUE_DATE,ACTIVE_CARD_COUNT
17244,PAXTON ASSOCIATES INC,2019-01-01,13.0
71739,STATE OF NEW YORK,2019-01-01,29151.0
149094,STUDIO IMPACT INC,2019-01-01,0.0
358416,PAXTON ASSOCIATES INC,2019-02-01,15.0
488754,STUDIO IMPACT INC,2019-02-01,0.0


In [12]:
print(len(df_v))
df_v['REVENUE_DATE'] = df_v.REVENUE_MONTH.astype(str) + "/01/" + df_v.REVENUE_YEAR.astype(str)
df_v['REVENUE_DATE'] = date_tz_naive(df_v['REVENUE_DATE'])
print(len(df_v))

df_v = df_v[['CUSTOMER','REVENUE_DATE', 'ACTIVE_CARD_COUNT']]

df_v_max = df_v[['CUSTOMER','ACTIVE_CARD_COUNT']]
df_max = df_v_max.groupby(by=["CUSTOMER"]).max().reset_index()
df_max.columns = ['CUSTOMER', 'ACTIVE_CARD_MAX']

print(len(df_v))
df_v.dropna(subset=['CUSTOMER'], inplace=True)
print(len(df_v))

df_v['REVENUE_DATE'] = pd.to_datetime(df_v['REVENUE_DATE'])
df_v.sort_values(['REVENUE_DATE'], inplace=True)

seen_accounts = df_v[df_v['ACTIVE_CARD_COUNT'] > 0].groupby(['CUSTOMER'], as_index=False)[['REVENUE_DATE']].first()
seen_accounts['FIRST_DATE'] = seen_accounts['REVENUE_DATE'] - pd.DateOffset(months=1)

df = df_v
period_end_date = end_date
match_type = 'program_flip'
period_start_date=None
split=None

drawdown = (100 - drawdown)/100
drawdown_fwd_check /= 100

inactive_date_start = pd.to_datetime(period_end_date) + relativedelta(months=-inactive_period)

df = df[df['REVENUE_DATE'] <= period_end_date].copy()

if period_start_date:
    period_start_date = pd.to_datetime(period_start_date)
    df = df[df['REVENUE_DATE'] >= period_start_date].copy()

all_customer_names = list(df['CUSTOMER'].unique())

if not split:
    split=1

all_customer_names_n = list(split_list(all_customer_names, split))

drop_df = pd.DataFrame()

165


AttributeError: 'DataFrame' object has no attribute 'REVENUE_MONTH'

2020-04-01    5
2020-09-01    5
2020-02-01    5
2020-01-01    5
2019-12-01    5
2020-06-01    5
2020-03-01    5
2020-10-01    5
2019-11-01    5
2020-07-01    4
2021-03-01    4
2021-04-01    4
2019-10-01    4
2020-12-01    4
2021-02-01    4
2019-05-01    4
2020-05-01    4
2019-06-01    4
2021-11-01    4
2020-11-01    4
2021-01-01    4
2019-09-01    4
2019-08-01    4
2020-08-01    4
2019-07-01    4
2021-10-01    4
2019-01-01    3
2021-07-01    3
2021-08-01    3
2021-09-01    3
2021-06-01    3
2021-05-01    3
2019-02-01    3
2022-01-01    3
2021-12-01    3
2019-04-01    3
2019-03-01    3
2022-04-01    3
2022-03-01    3
2022-02-01    3
2022-07-01    2
2022-08-01    2
2022-09-01    2
2022-05-01    2
2022-06-01    2
Name: REVENUE_DATE, dtype: int64

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

#CALCULATED_DRAW_DOWNS_df = NAFCUSTOMER_ACTIVE_CARDS_FULL_df # For this sample code, simply copy input to output


# Write recipe outputs
#CALCULATED_DRAW_DOWNS = dataiku.Dataset("CALCULATED_DRAW_DOWNS")
#CALCULATED_DRAW_DOWNS.write_with_schema(CALCULATED_DRAW_DOWNS_df)